In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import re

In [2]:
#Rive Droite = 884
#Chateau = 885
#Chantiers = 886
#Porchefontaine = 679
#Montreuil = 566
stif_code = {
    "Versailles Rive Droite" : "884",
    "Versailles Château Rive Gauche" : "885",
    "Versailles Chantiers" : "886",
    "Porchefontaine" : "679",
    "Montreuil" : "566"
}

stif_code_reverse = {value:key for key, value in stif_code.items()}

In [3]:
test = pd.read_csv('ef_versailles.csv')

In [4]:
test.head()

,Station_name,Network,Longitude,Latitude,City,Traffic_2015,Traffic_2016,Traffic_2017,Traffic_2018,Traffic_2019,Traffic_2020,Traffic_2021,Postcode,Traffic_2022,dept,Traffic_pre_cov,Traffic_pre_cov_sd,covid_drop_2020,covid_drop_2021,Traffic_2020_2021,Origin_id,Rank_rer_2015,Rank_rer_2016,Rank_rer_2017,Rank_rer_2018,Rank_rer_2019,Rank_rer_2020,Rank_rer_2021,Rank_rer_2022
0,Versailles Chantiers,RER,2.135160,48.795690,versailles,24191310.0,24132107.0,23974181.0,23336771.0,22909416.0,12376593,15922909,78000.0,18934500.0,78.0,23708757.0,501966.560353,-0.477974,-0.328395,0.286534,531,8.0,9.0,9.0,10.0,10.0,8.0,11.0,8.0
1,Versailles Château Rive Gauche,RER,2.128880,48.800330,versailles,8748796.0,8367046.0,8009738.0,7520468.0,7015859.0,2886249,3310046,78000.0,5795124.0,78.0,7932381.4,611741.140672,-0.636143,-0.582717,0.146833,457,41.0,49.0,55.0,59.0,64.0,71.0,103.0,62.0
2,Versailles Rive Droite,RER,2.135087,48.809482,versailles,5357929.0,5327810.0,5562438.0,5705373.0,5855418.0,2039014,4698156,78000.0,5530365.0,78.0,5561793.6,201563.810161,-0.633389,-0.155280,1.304131,458,83.0,85.0,81.0,81.0,79.0,109.0,71.0,64.0
3,Montreuil (transilien),RER,2.152993,48.805941,versailles,2248554.0,2137642.0,2192249.0,2204405.0,2226698.0,748793,1897965,78000.0,2010237.0,78.0,2201909.6,37470.103948,-0.659935,-0.138037,1.534699,423,183.0,198.0,196.0,196.0,194.0,231.0,178.0,156.0
4,Porchefontaine,RER,2.152379,48.796528,versailles,956155.0,946641.0,933001.0,902371.0,875987.0,398498,545751,78000.0,723565.0,78.0,922831.0,29643.906328,-0.568179,-0.408612,0.369520,663,264.0,267.0,273.0,275.0,279.0,276.0,301.0,241.0


In [5]:
traffic = pd.read_csv('ef_versailles.csv')
traffic = traffic.drop(["Network", "Longitude", 'Latitude', "City", "Postcode", "dept"], axis = 1)
traffic = traffic.rename(columns = {excol:str(excol).lower() for excol in traffic.columns})
traffic[["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd"]] = traffic[["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd"]].astype('int')
traffic[["rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]] = traffic[["rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]].astype('int')
traffic.loc[traffic["origin_id"] == 531, "station_name"] = "Versailles Chantiers"
traffic.loc[traffic["origin_id"] == 457, "station_name"] = "Versailles Château Rive Gauche"
traffic.loc[traffic["origin_id"] == 458, "station_name"] = "Versailles Rive Droite"
traffic.loc[traffic["origin_id"] == 423, "station_name"] = "Montreuil"
traffic.loc[traffic["origin_id"] == 663, "station_name"] = "Porchefontaine"
traffic['station_id'] = traffic['station_name'].map(stif_code)
traffic = traffic[["station_name", "station_id", "origin_id", "traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd", "covid_drop_2020","covid_drop_2021","traffic_2020_2021", "rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]]
traffic = traffic.drop(["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd", "covid_drop_2020","covid_drop_2021","traffic_2020_2021"], axis = 1)
traffic = traffic.rename(columns = {"rank_rer_2015" : 2015, "rank_rer_2016" : 2016, "rank_rer_2017" : 2017, "rank_rer_2018" : 2018, "rank_rer_2019" : 2019, "rank_rer_2020" : 2020, "rank_rer_2021" : 2021, "rank_rer_2022" : 2022})
traffic = pd.melt(traffic, id_vars = ['station_name', 'station_id', 'origin_id'], var_name = "year", value_name = "rank")
traffic.head()

,station_name,station_id,origin_id,year,rank
0,Versailles Chantiers,886,531,2015,8
1,Versailles Château Rive Gauche,885,457,2015,41
2,Versailles Rive Droite,884,458,2015,83
3,Montreuil,566,423,2015,183
4,Porchefontaine,679,663,2015,264


In [6]:
## EVERY STATION 

# 2015

df15_s1 = pd.read_csv('original_datasets/data-rf-2015/2015S1_NB_FER.csv', sep = ";")
df15_s2 = pd.read_csv('original_datasets/data-rf-2015/2015S2_NB_FER.csv', sep = ";")

# 2016

df16_s1 = pd.read_csv('original_datasets/data-rf-2016/2016S1_NB_FER.txt', sep = "\t")
df16_s2 = pd.read_csv('original_datasets/data-rf-2016/2016S2_NB_FER.txt', sep = "\t")

# 2017

df17_s1 = pd.read_csv('original_datasets/data-rf-2017/2017S1_NB_FER.txt', sep = "\t")
df17_s2 = pd.read_csv('original_datasets/data-rf-2017/2017_S2_NB_FER.txt', sep = "\t")

# 2018

df18_s1 = pd.read_csv('original_datasets/data-rf-2018/2018_S1_NB_FER.txt', sep = "\t")
df18_s2 = pd.read_csv('original_datasets/data-rf-2018/2018_S2_NB_FER.txt', sep = "\t")

# 2019

df19_s1 = pd.read_csv('original_datasets/data-rf-2019/2019_S1_NB_FER.txt', sep = "\t")
df19_s2 = pd.read_csv('original_datasets/data-rf-2019/2019_S2_NB_FER.txt', sep = "\t")

# 2020

df20_s1 = pd.read_csv('original_datasets/data-rf-2020/2020_S1_NB_FER.txt', sep = "\t")
df20_s2 = pd.read_csv('original_datasets/data-rf-2020/2020_S2_NB_FER.txt', sep = "\t")

# 2021

df21_s1 = pd.read_csv('original_datasets/data-rf-2021/2021_S1_NB_FER.txt', sep = "\t")
df21_s2 = pd.read_csv('original_datasets/data-rf-2021/2021_S2_NB_FER.txt', sep = "\t")

# 2022

df22_s1 = pd.read_csv('original_datasets/data-rf-2022/2022_S1_NB_FER.txt', sep = "\t")
df22_s2 = pd.read_csv('original_datasets/data-rf-2022/2022_S2_NB_FER.txt', sep = ";")

# 2023

df23_s1_all = pd.read_csv('original_datasets/data-rf-2023/2023_S1_NB_FER.csv', sep = ";")

df2_all = pd.concat([df15_s1, df15_s2, df16_s1, df16_s2, df17_s1, df17_s2, df18_s1, df18_s2,
                df19_s1, df19_s2, df20_s1, df20_s2, df21_s1, df21_s2, df22_s1, df22_s2,
               ]).reset_index(drop = True)

df2_all['JOUR'] = pd.to_datetime(df2_all['JOUR'], format = '%d/%m/%Y')
df23_s1_all['JOUR'] = pd.to_datetime(df23_s1_all['JOUR'], format = '%Y-%m-%d')


In [7]:
network_dict = {
    '110' : "metro", 
    '803' : "RER C", 
    '854' : "L & J", 
    '804' : "RER D", 
    '801' : "RER A", 
    '802' : "RER B", 
    '853' : "H", 
    '852' : "N", 
    '850' : "P", 
    '805' : "RER E",
    '851' : "R", 
    '822' : "T4"
}

In [8]:
daily_all = pd.concat([df2_all, df23_s1_all]).reset_index(drop = True)
daily_all = daily_all[daily_all["CODE_STIF_RES"] != "110"]
daily_all = daily_all.drop(["CODE_STIF_TRNS", "ID_REFA_LDA", "lda"], axis = 1)
daily_all = daily_all.rename(columns = {"JOUR" : "date", "CODE_STIF_RES" : "network",  "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
daily_all['network'] = daily_all['network'].map(network_dict)
daily_all['n_passengers'] = daily_all.apply(lambda row: 2 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)
daily_all['n_passengers'] = daily_all['n_passengers'].astype("int")
daily_all['year'] = daily_all['date'].dt.year
daily_all['day'] = daily_all['date'].dt.day
daily_all['month'] = daily_all['date'].dt.month
daily_all['day_name'] = daily_all['date'].dt.day_name()
daily_all['month_name'] = daily_all['date'].dt.month_name()
daily_all = daily_all.pivot_table(index=['date', 'station_id', 'station_name', 'year', 'day', 'month', 'day_name', 'month_name', 'network'],
                                  columns='pass_type', values='n_passengers', aggfunc='sum').reset_index()
daily_all.columns.name = None
daily_all[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]] = daily_all[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]].fillna(0).astype("int")
daily_all['social'] = daily_all['FGT'] + daily_all['TST']
daily_all['other'] = daily_all['?'] + daily_all['AUTRE TITRE'] + daily_all['NON DEFINI']
daily_all = daily_all.drop(['?', "AUTRE TITRE", "FGT", "NON DEFINI", "TST"], axis = 1)
daily_all = daily_all.rename(columns = {"AMETHYSTE" : "amethyste", "IMAGINE R" : "imagine_r", "NAVIGO": "navigo", "NAVIGO JOUR": "navigo_jour"})
daily_all["daily_traffic"] = daily_all["amethyste"] + daily_all["imagine_r"] + daily_all["navigo"] + daily_all["navigo_jour"] + daily_all ["social"] + daily_all["other"]

S1 = ["January", "February", "March", "April", "May", "June"]
S2 = ["July", "August", "September", "October", "November", "December"]

daily_all["semester"] = daily_all.apply(lambda row: 1 if row['month_name'] in S1 else 2 if row['month_name'] in S2 else None, axis = 1)
daily_all = daily_all[daily_all['network'] != "T4"]
daily_all.head(10)

,date,station_id,station_name,year,day,month,day_name,month_name,network,amethyste,imagine_r,navigo,navigo_jour,social,other,daily_traffic,semester
0,2015-01-01,101,BOURAY,2015,1,1,Thursday,January,RER C,2,26,17,0,4,2,51,1
1,2015-01-01,102,BOURG-LA-REINE,2015,1,1,Thursday,January,RER B,81,423,977,0,243,94,1818,1
2,2015-01-01,103,BOURRON-MARLOTTE-GREZ,2015,1,1,Thursday,January,R,0,2,2,0,2,0,6,1
3,2015-01-01,105,BOUSSY-SAINT-ANTOINE,2015,1,1,Thursday,January,RER D,2,13,30,0,18,2,65,1
4,2015-01-01,106,BOUTIGNY-SUR-ESSONNE,2015,1,1,Thursday,January,RER D,0,7,9,0,2,2,20,1
5,2015-01-01,108,BRETIGNY,2015,1,1,Thursday,January,RER C,32,196,311,0,132,33,704,1
6,2015-01-01,109,BREUILLET-BRUYERES-LE-CHATEL,2015,1,1,Thursday,January,RER C,0,9,7,0,2,0,18,1
7,2015-01-01,110,BREUILLET-VILLAGE,2015,1,1,Thursday,January,RER C,5,9,20,0,7,0,41,1
8,2015-01-01,111,BREVAL,2015,1,1,Thursday,January,L & J,0,5,6,0,0,0,11,1
9,2015-01-01,114,BRUNOY,2015,1,1,Thursday,January,RER D,31,132,326,0,155,16,660,1


<h4>All stations</h4>

In [9]:
means_all = daily_all.groupby(['year'])["daily_traffic"].mean().reset_index()
means_all['daily_traffic'] = means_all['daily_traffic'].astype('int')
means_all = means_all.rename(columns = {"daily_traffic" : "mean"})

means_wd = daily_all[~daily_all['day_name'].isin(['Saturday', 'Sunday'])].groupby(['year'])["daily_traffic"].mean().reset_index()
means_wd['daily_traffic'] = means_wd['daily_traffic'].astype('int')
means_wd = means_wd.rename(columns = {"daily_traffic" : "mean_weekday"})

means_we = daily_all[daily_all['day_name'].isin(['Saturday', 'Sunday'])].groupby(['year'])["daily_traffic"].mean().reset_index()
means_we['daily_traffic'] = means_we['daily_traffic'].astype('int')
means_we = means_we.rename(columns = {"daily_traffic" : "mean_weekend"})

medians_all = daily_all.groupby(['year'])["daily_traffic"].median().reset_index()
medians_all['daily_traffic'] = medians_all['daily_traffic'].astype('int')
medians_all = medians_all.rename(columns = {"daily_traffic" : "median"})

medians_wd = daily_all[~daily_all['day_name'].isin(['Saturday', 'Sunday'])].groupby(['year'])["daily_traffic"].median().reset_index()
medians_wd['daily_traffic'] = medians_wd['daily_traffic'].astype('int')
medians_wd = medians_wd.rename(columns = {"daily_traffic" : "median_weekday"})

medians_we = daily_all[daily_all['day_name'].isin(['Saturday', 'Sunday'])].groupby(['year'])["daily_traffic"].median().reset_index()
medians_we['daily_traffic'] = medians_we['daily_traffic'].astype('int')
medians_we = medians_we.rename(columns = {"daily_traffic" : "median_weekend"})

yearly_all = means_all.merge(means_wd, on = "year").merge(means_we, on = "year").merge(medians_all, on = "year").merge(medians_wd, on = "year").merge(medians_we, on = "year")
yearly_all

,year,mean,mean_weekday,mean_weekend,median,median_weekday,median_weekend
0,2015,3605,4330,1768,946,1354,427
1,2016,3958,4740,1999,1156,1579,536
2,2017,4285,5133,2169,1366,1854,606
3,2018,4296,5091,2256,1298,1754,607
4,2019,4600,5472,2381,1408,1973,633
5,2020,2665,3156,1393,573,781,257
6,2021,3376,3951,1913,1073,1444,508
7,2022,4264,4982,2465,1403,1871,669
8,2023,4442,5161,2585,1458,1917,721


<h4>Lines N and U</h4>

In [10]:
means_all = daily_all[daily_all['network'] == "N"].groupby(['year'])["daily_traffic"].mean().reset_index()
means_all['daily_traffic'] = means_all['daily_traffic'].astype('int')
means_all = means_all.rename(columns = {"daily_traffic" : "mean"})

means_wd = daily_all[(~daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "N")].groupby(['year'])["daily_traffic"].mean().reset_index()
means_wd['daily_traffic'] = means_wd['daily_traffic'].astype('int')
means_wd = means_wd.rename(columns = {"daily_traffic" : "mean_weekday"})

means_we = daily_all[(daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "N")].groupby(['year'])["daily_traffic"].mean().reset_index()
means_we['daily_traffic'] = means_we['daily_traffic'].astype('int')
means_we = means_we.rename(columns = {"daily_traffic" : "mean_weekend"})

medians_all = daily_all[daily_all['network'] == "N"].groupby(['year'])["daily_traffic"].median().reset_index()
medians_all['daily_traffic'] = medians_all['daily_traffic'].astype('int')
medians_all = medians_all.rename(columns = {"daily_traffic" : "median"})

medians_wd = daily_all[(~daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "N")].groupby(['year'])["daily_traffic"].median().reset_index()
medians_wd['daily_traffic'] = medians_wd['daily_traffic'].astype('int')
medians_wd = medians_wd.rename(columns = {"daily_traffic" : "median_weekday"})

medians_we = daily_all[(daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "N")].groupby(['year'])["daily_traffic"].median().reset_index()
medians_we['daily_traffic'] = medians_we['daily_traffic'].astype('int')
medians_we = medians_we.rename(columns = {"daily_traffic" : "median_weekend"})

yearly_n = means_all.merge(means_wd, on = "year").merge(means_we, on = "year").merge(medians_all, on = "year").merge(medians_wd, on = "year").merge(medians_we, on = "year")
yearly_n

,year,mean,mean_weekday,mean_weekend,median,median_weekday,median_weekend
0,2015,2572,3159,1071,530,697,249
1,2016,2710,3324,1165,708,974,323
2,2017,2830,3506,1132,777,1011,251
3,2018,2675,3273,1127,677,935,256
4,2019,2888,3518,1290,826,1090,343
5,2020,1688,2030,802,330,459,168
6,2021,1812,2176,886,573,772,276
7,2022,2348,2833,1127,723,948,350
8,2023,2467,2931,1264,755,968,413


<h4>Line L</h4>

In [11]:
stations_L = ["PONT-CARDINET", "ASNIERES", "LA DEFENSE-GRANDE ARCHE", "SAINT-CLOUD", "SURESNES-MONT VALERIEN",
              "CLICHY-LEVALLOIS", "SEVRES-VILLE-D'AVRAY", "VIROFLAY RIVE DROITE", "CHAVILLE RIVE DROITE",
              "VERSAILLES RIVE DROITE", "SAINT-LAZARE", "SEVRES VILLE", "ST CLOUD", "SURESNES", "VERSAILLES RD",
              "VIROFLAY RD", "CHAVILLE RD", "CLICHY LEVALLOI", "ASNIERES SUR S", "DEFENSE"]

means_all = daily_all[(daily_all['network'] == "L & J") & (daily_all['station_name'].isin(stations_L))].groupby(['year'])["daily_traffic"].mean().reset_index()
means_all['daily_traffic'] = means_all['daily_traffic'].astype('int')
means_all = means_all.rename(columns = {"daily_traffic" : "mean"})

means_wd = daily_all[(~daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "L & J") & (daily_all['station_name'].isin(stations_L))].groupby(['year'])["daily_traffic"].mean().reset_index()
means_wd['daily_traffic'] = means_wd['daily_traffic'].astype('int')
means_wd = means_wd.rename(columns = {"daily_traffic" : "mean_weekday"})

means_we = daily_all[(daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "L & J") & (daily_all['station_name'].isin(stations_L))].groupby(['year'])["daily_traffic"].mean().reset_index()
means_we['daily_traffic'] = means_we['daily_traffic'].astype('int')
means_we = means_we.rename(columns = {"daily_traffic" : "mean_weekend"})

medians_all = daily_all[(daily_all['network'] == "L & J") & (daily_all['station_name'].isin(stations_L))].groupby(['year'])["daily_traffic"].median().reset_index()
medians_all['daily_traffic'] = medians_all['daily_traffic'].astype('int')
medians_all = medians_all.rename(columns = {"daily_traffic" : "median"})

medians_wd = daily_all[(~daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "L & J") & (daily_all['station_name'].isin(stations_L))].groupby(['year'])["daily_traffic"].median().reset_index()
medians_wd['daily_traffic'] = medians_wd['daily_traffic'].astype('int')
medians_wd = medians_wd.rename(columns = {"daily_traffic" : "median_weekday"})

medians_we = daily_all[(daily_all['day_name'].isin(['Saturday', 'Sunday'])) & (daily_all['network'] == "L & J") & (daily_all['station_name'].isin(stations_L))].groupby(['year'])["daily_traffic"].median().reset_index()
medians_we['daily_traffic'] = medians_we['daily_traffic'].astype('int')
medians_we = medians_we.rename(columns = {"daily_traffic" : "median_weekend"})

yearly_l = means_all.merge(means_wd, on = "year").merge(means_we, on = "year").merge(medians_all, on = "year").merge(medians_wd, on = "year").merge(medians_we, on = "year")
yearly_l

,year,mean,mean_weekday,mean_weekend,median,median_weekday,median_weekend
0,2015,4601,5697,1847,2015,2729,751
1,2016,5375,6656,2186,2449,3467,964
2,2017,6130,7600,2465,3244,4142,1220
3,2018,7239,8959,2906,3517,4593,1471
4,2019,11535,13993,5144,3935,5545,1642
5,2020,8910,10727,4293,1974,2674,874
6,2021,10876,12782,6045,3094,4119,1626
7,2022,13421,15902,7268,4401,6095,2356
8,2023,14176,16833,7406,4755,6603,2671


In [12]:
yearly_all.to_csv('yearly_all.csv', index = False)
yearly_n.to_csv('yearly_n.csv', index = False)
yearly_l.to_csv('yearly_l.csv', index = False)

<h4>Versailles</h4>

In [13]:
# 2015

df15_s1 = pd.read_csv('original_datasets/data-rf-2015/2015S1_NB_FER.csv', sep = ";")
df15_s1 = df15_s1[df15_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df15_s2 = pd.read_csv('original_datasets/data-rf-2015/2015S2_NB_FER.csv', sep = ";")
df15_s2 = df15_s2[df15_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2016

df16_s1 = pd.read_csv('original_datasets/data-rf-2016/2016S1_NB_FER.txt', sep = "\t")
df16_s1 = df16_s1[df16_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df16_s2 = pd.read_csv('original_datasets/data-rf-2016/2016S2_NB_FER.txt', sep = "\t")
df16_s2 = df16_s2[df16_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2017

df17_s1 = pd.read_csv('original_datasets/data-rf-2017/2017S1_NB_FER.txt', sep = "\t")
df17_s1 = df17_s1[df17_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df17_s2 = pd.read_csv('original_datasets/data-rf-2017/2017_S2_NB_FER.txt', sep = "\t")
df17_s2 = df17_s2[df17_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2018

df18_s1 = pd.read_csv('original_datasets/data-rf-2018/2018_S1_NB_FER.txt', sep = "\t")
df18_s1 = df18_s1[df18_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df18_s2 = pd.read_csv('original_datasets/data-rf-2018/2018_S2_NB_FER.txt', sep = "\t")
df18_s2 = df18_s2[df18_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2019

df19_s1 = pd.read_csv('original_datasets/data-rf-2019/2019_S1_NB_FER.txt', sep = "\t")
df19_s1 = df19_s1[df19_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df19_s2 = pd.read_csv('original_datasets/data-rf-2019/2019_S2_NB_FER.txt', sep = "\t")
df19_s2 = df19_s2[df19_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2020

df20_s1 = pd.read_csv('original_datasets/data-rf-2020/2020_S1_NB_FER.txt', sep = "\t")
df20_s1 = df20_s1[df20_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df20_s2 = pd.read_csv('original_datasets/data-rf-2020/2020_S2_NB_FER.txt', sep = "\t")
df20_s2 = df20_s2[df20_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2021

df21_s1 = pd.read_csv('original_datasets/data-rf-2021/2021_S1_NB_FER.txt', sep = "\t")
df21_s1 = df21_s1[df21_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df21_s2 = pd.read_csv('original_datasets/data-rf-2021/2021_S2_NB_FER.txt', sep = "\t")
df21_s2 = df21_s2[df21_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2022

df22_s1 = pd.read_csv('original_datasets/data-rf-2022/2022_S1_NB_FER.txt', sep = "\t")
df22_s1 = df22_s1[df22_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df22_s2 = pd.read_csv('original_datasets/data-rf-2022/2022_S2_NB_FER.txt', sep = ";")
df22_s2 = df22_s2[df22_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2023

df23_s1 = pd.read_csv('original_datasets/data-rf-2023/2023_S1_NB_FER.csv', sep = ";")
df23_s1 = df23_s1[df23_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df2 = pd.concat([df15_s1, df15_s2, df16_s1, df16_s2, df17_s1, df17_s2, df18_s1, df18_s2,
                df19_s1, df19_s2, df20_s1, df20_s2, df21_s1, df21_s2, df22_s1, df22_s2,
               ]).reset_index(drop = True)

df2['JOUR'] = pd.to_datetime(df2['JOUR'], format = '%d/%m/%Y')
df23_s1['JOUR'] = pd.to_datetime(df23_s1['JOUR'], format = '%Y-%m-%d')



In [14]:
daily = pd.concat([df2, df23_s1]).reset_index(drop = True)
daily = daily.drop(["CODE_STIF_TRNS", "CODE_STIF_RES", "ID_REFA_LDA", "lda"], axis = 1)
daily = daily.rename(columns = {"JOUR" : "date", "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
daily['station_name'] = daily['station_id'].map(stif_code_reverse)
daily['n_passengers'] = daily.apply(lambda row: 2 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)
daily['n_passengers'] = daily['n_passengers'].astype("int")
daily['year'] = daily['date'].dt.year
daily['day'] = daily['date'].dt.day
daily['month'] = daily['date'].dt.month
daily['day_name'] = daily['date'].dt.day_name()
daily['month_name'] = daily['date'].dt.month_name()
daily = daily.pivot_table(index=['date', 'station_id', 'station_name', 'year', 'day', 'month', 'day_name', 'month_name'],
                                  columns='pass_type', values='n_passengers', aggfunc='sum').reset_index()
daily.columns.name = None
daily[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]] = daily[["?", "AMETHYSTE", "AUTRE TITRE", "FGT", "IMAGINE R", "NAVIGO", "NAVIGO JOUR", "NON DEFINI", "TST"]].fillna(0).astype("int")
daily['social'] = daily['FGT'] + daily['TST']
daily['other'] = daily['?'] + daily['AUTRE TITRE'] + daily['NON DEFINI']
daily = daily.drop(['?', "AUTRE TITRE", "FGT", "NON DEFINI", "TST"], axis = 1)
daily = daily.rename(columns = {"AMETHYSTE" : "amethyste", "IMAGINE R" : "imagine_r", "NAVIGO": "navigo", "NAVIGO JOUR": "navigo_jour"})
daily["daily_traffic"] = daily["amethyste"] + daily["imagine_r"] + daily["navigo"] + daily["navigo_jour"] + daily ["social"] + daily["other"]

S1 = ["January", "February", "March", "April", "May", "June"]
S2 = ["July", "August", "September", "October", "November", "December"]

daily["semester"] = daily.apply(lambda row: 1 if row['month_name'] in S1 else 2 if row['month_name'] in S2 else None, axis = 1)

daily.head(50)

,date,station_id,station_name,year,day,month,day_name,month_name,amethyste,imagine_r,navigo,navigo_jour,social,other,daily_traffic,semester
0,2015-01-01,566,Montreuil,2015,1,1,Thursday,January,16,77,111,0,22,9,235,1
1,2015-01-01,679,Porchefontaine,2015,1,1,Thursday,January,7,33,52,0,4,2,98,1
2,2015-01-01,884,Versailles Rive Droite,2015,1,1,Thursday,January,31,204,463,0,89,39,826,1
3,2015-01-01,885,Versailles Château Rive Gauche,2015,1,1,Thursday,January,28,133,398,0,67,21,647,1
4,2015-01-01,886,Versailles Chantiers,2015,1,1,Thursday,January,35,500,751,0,157,123,1566,1
5,2015-01-02,566,Montreuil,2015,2,1,Friday,January,23,182,673,0,117,22,1017,1
6,2015-01-02,679,Porchefontaine,2015,2,1,Friday,January,11,101,270,0,22,13,417,1
7,2015-01-02,884,Versailles Rive Droite,2015,2,1,Friday,January,66,522,1760,0,261,98,2707,1
8,2015-01-02,885,Versailles Château Rive Gauche,2015,2,1,Friday,January,51,313,1270,0,171,125,1930,1
9,2015-01-02,886,Versailles Chantiers,2015,2,1,Friday,January,119,1281,3738,0,596,359,6093,1


In [15]:
gares = pd.read_csv('original_datasets/referentiel-gares-voyageurs.csv', sep = ";")
gares["Commune"] = gares["Commune"].dropna()
infos_v = gares[gares["Commune"] == 'Versailles'][["Longitude", 'Latitude', "Intitulé gare"]]
infos_v['stif_code'] = infos_v['Intitulé gare'].map(stif_code)
infos_v = infos_v.rename(columns = {"Longitude" : "longitude", "Latitude" : "latitude", "Intitulé gare" : "station_name", "stif_code" : "station_id"})
infos_v = infos_v.reset_index(drop = True)
infos_v = infos_v[['station_name', 'station_id', 'latitude', 'longitude']]

In [16]:
infos_v.head()

,station_name,station_id,latitude,longitude
0,Montreuil,566,48.806732,2.151169
1,Versailles Rive Droite,884,48.809652,2.134752
2,Porchefontaine,679,48.796938,2.154016
3,Versailles Château Rive Gauche,885,48.800364,2.128827
4,Versailles Chantiers,886,48.795826,2.135883


In [17]:
count_code = {
    "Versailles Rive Droite" : 87382861,
    "Versailles Château Rive Gauche" : 87393157,
    "Versailles Chantiers" : 87393009,
    "Porchefontaine" : 87393165,
    "Montreuil" : 87382879
}

In [18]:
count = pd.read_csv('original_datasets/comptage-transilien.csv', sep = ";")
count = count[count['Code Gare'].isin(count_code.values())]
count = count.rename(columns = {"Nom gare" : "station_name", "Code Gare" : "station_id", "Type jour" : "day_type", "Année" : "year", "Date de comptage" : "date", "Ligne" : "line", "Tranche horaire" : "time_period", "Montants" : "counts"})
count = count.drop(["Axe", "year", "date"], axis = 1)
count['station_name'] = count['station_id'].map({value:key for key,value in count_code.items()})
count['station_id'] = count['station_name'].map(stif_code)
count['day_type'] = count['day_type'].map({"JOB":'weekday', "Samedi": "saturday", "Dimanche":"sunday"})
count['time_period'] = count['time_period'].map({"Avant 6h" : "before 6am", "De 6h à 10h": "6am - 10am", "De 10h à 16h": "10am - 4pm", "De 16h à 20h":"4pm - 8pm" ,"Après 20h" : "after 8pm"})
count.head(50)

,station_name,station_id,day_type,line,time_period,counts
218,Porchefontaine,679,sunday,C,after 8pm,46
349,Porchefontaine,679,weekday,C,10am - 4pm,337
359,Porchefontaine,679,saturday,C,after 8pm,63
375,Versailles Chantiers,886,sunday,C,10am - 4pm,724
586,Porchefontaine,679,saturday,C,before 6am,7
652,Porchefontaine,679,saturday,C,4pm - 8pm,249
726,Versailles Chantiers,886,saturday,C,4pm - 8pm,1015
759,Porchefontaine,679,sunday,C,6am - 10am,62
773,Versailles Chantiers,886,saturday,C,10am - 4pm,1105
782,Versailles Chantiers,886,weekday,C,10am - 4pm,1343


In [19]:
traffic.to_csv('ef_v_traffic.csv', index = False)
daily.to_csv('ef_v_daily.csv', index = False)
infos_v.to_csv('ef_v_infos.csv', index = False)
count.to_csv('ef_v_count.csv', index = False)
daily_all.to_csv('ef_v_dailly_all', index = False)

In [20]:
traffic.head()

,station_name,station_id,origin_id,year,rank
0,Versailles Chantiers,886,531,2015,8
1,Versailles Château Rive Gauche,885,457,2015,41
2,Versailles Rive Droite,884,458,2015,83
3,Montreuil,566,423,2015,183
4,Porchefontaine,679,663,2015,264


In [21]:
daily.head()

,date,station_id,station_name,year,day,month,day_name,month_name,amethyste,imagine_r,navigo,navigo_jour,social,other,daily_traffic,semester
0,2015-01-01,566,Montreuil,2015,1,1,Thursday,January,16,77,111,0,22,9,235,1
1,2015-01-01,679,Porchefontaine,2015,1,1,Thursday,January,7,33,52,0,4,2,98,1
2,2015-01-01,884,Versailles Rive Droite,2015,1,1,Thursday,January,31,204,463,0,89,39,826,1
3,2015-01-01,885,Versailles Château Rive Gauche,2015,1,1,Thursday,January,28,133,398,0,67,21,647,1
4,2015-01-01,886,Versailles Chantiers,2015,1,1,Thursday,January,35,500,751,0,157,123,1566,1


In [22]:
infos_v.head()

,station_name,station_id,latitude,longitude
0,Montreuil,566,48.806732,2.151169
1,Versailles Rive Droite,884,48.809652,2.134752
2,Porchefontaine,679,48.796938,2.154016
3,Versailles Château Rive Gauche,885,48.800364,2.128827
4,Versailles Chantiers,886,48.795826,2.135883


In [23]:
count.head()

,station_name,station_id,day_type,line,time_period,counts
218,Porchefontaine,679,sunday,C,after 8pm,46
349,Porchefontaine,679,weekday,C,10am - 4pm,337
359,Porchefontaine,679,saturday,C,after 8pm,63
375,Versailles Chantiers,886,sunday,C,10am - 4pm,724
586,Porchefontaine,679,saturday,C,before 6am,7
